In [1]:
import tensorflow as tf
import numpy as np
import scipy

In [2]:
import tensorflow as tf
import numpy as np
from scipy.optimize import minimize
from tensorflow.keras.optimizers import Adam

class PhysicsInformedNN:
    # Initialize the class
    def __init__(self, x, y, t, u, v, layers):

        X = np.concatenate([x, y, t], 1)

        self.lb = X.min(0)
        self.ub = X.max(0)

        self.X = X

        self.x = tf.convert_to_tensor(X[:, 0:1], dtype=tf.float32)
        self.y = tf.convert_to_tensor(X[:, 1:2], dtype=tf.float32)
        self.t = tf.convert_to_tensor(X[:, 2:3], dtype=tf.float32)

        self.u = tf.convert_to_tensor(u, dtype=tf.float32)
        self.v = tf.convert_to_tensor(v, dtype=tf.float32)

        self.layers = layers

        # Initialize NN
        self.model = self.create_model(layers)

        # Initialize parameters
        self.lambda_1 = tf.Variable([0.0], dtype=tf.float32)
        self.lambda_2 = tf.Variable([0.0], dtype=tf.float32)

    def create_model(self, layers):
        model = tf.keras.Sequential()
        for i in range(len(layers)-2):
            model.add(tf.keras.layers.Dense(layers[i+1], activation=tf.nn.tanh,
                                            input_shape=(layers[i],),
                                            kernel_initializer='glorot_normal'))
        model.add(tf.keras.layers.Dense(layers[-1], kernel_initializer='glorot_normal'))
        return model

    def neural_net(self, X):
        return self.model(X)

    def loss(self, u_pred, v_pred, f_u_pred, f_v_pred):
        return tf.reduce_sum(tf.square(self.u - u_pred)) + \
               tf.reduce_sum(tf.square(self.v - v_pred)) + \
               tf.reduce_sum(tf.square(f_u_pred)) + \
               tf.reduce_sum(tf.square(f_v_pred))

    def net_NS(self, x, y, t):
        lambda_1 = self.lambda_1
        lambda_2 = self.lambda_2

        with tf.GradientTape(persistent=True) as tape2:
            tape2.watch([x, y, t])
            with tf.GradientTape(persistent=True) as tape1:
                tape1.watch([x, y, t])
                psi_and_p = self.neural_net(tf.concat([x, y, t], 1))
                psi = psi_and_p[:, 0:1]
                p = psi_and_p[:, 1:2]
                u = tape1.gradient(psi, t)
                v = -tape1.gradient(psi, x)
            u_x = tape1.gradient(u, x)
            u_y = tape1.gradient(u, y)
            v_x = tape1.gradient(v, x)
            v_y = tape1.gradient(v, y)
            u_t = tape2.gradient(u, t)
            v_t = tape2.gradient(v, t)
            u_xx = tape2.gradient(u_x, x)
            u_yy = tape2.gradient(u_y, y)
            v_xx = tape2.gradient(v_x, x)
            v_yy = tape2.gradient(v_y, y)
        p_x = tape2.gradient(p, x)
        p_y = tape2.gradient(p, y)

        f_u = u_t + lambda_1 * (u * u_x + v * u_y) + p_x - lambda_2 * (u_xx + u_yy)
        f_v = v_t + lambda_1 * (u * v_x + v * v_y) + p_y - lambda_2 * (v_xx + v_yy)

        return u, v, p, f_u, f_v



    def callback(self, loss, lambda_1, lambda_2):
        print('Loss: %.3e, l1: %.3f, l2: %.5f' % (loss, lambda_1, lambda_2))

    # def train(self, nIter):
    #     optimizer = Adam()

    #     for it in range(nIter):
    #         with tf.GradientTape() as tape:
    #             u_pred, v_pred, p_pred, f_u_pred, f_v_pred = self.net_NS(self.x, self.y, self.t)
    #             loss_value = self.loss(u_pred, v_pred, f_u_pred, f_v_pred)

    #         grads = tape.gradient(loss_value, self.model.trainable_variables)
    #         optimizer.apply_gradients(zip(grads, self.model.trainable_variables))

    #         # Print
    #         if it % 10 == 0:
    #             lambda_1_value = self.lambda_1.numpy()
    #             lambda_2_value = self.lambda_2.numpy()
    #             print('It: %d, Loss: %.3e, l1: %.3f, l2: %.5f' %
    #                   (it, loss_value, lambda_1_value, lambda_2_value))

    #     # SciPy optimization after Adam optimization
    #     method = 'L-BFGS-B'
    #     options = {'maxiter': 50000,
    #                'maxfun': 50000,
    #                'maxcor': 50,
    #                'maxls': 50,
    #                'ftol': 1.0 * np.finfo(float).eps}

    #     def loss_function(variables):
    #         self.lambda_1.assign([variables[0]])
    #         self.lambda_2.assign([variables[1]])
    #         u_pred, v_pred, p_pred, f_u_pred, f_v_pred = self.net_NS(self.x, self.y, self.t)
    #         loss_value = self.loss(u_pred, v_pred, f_u_pred, f_v_pred)
    #         return loss_value.numpy()

    #     def loss_gradient(variables):
    #         with tf.GradientTape() as tape:
    #             loss_value = loss_function(variables)
    #         grad = tape.gradient(loss_value, [self.lambda_1, self.lambda_2])
    #         return tf.convert_to_tensor([grad[0], grad[1]])

    #     variables = tf.Variable([0.0, 0.0], dtype=tf.float32)
    #     result = minimize(loss_function, variables, method=method, jac=loss_gradient, options=options)
    #     self.lambda_1.assign([result.x[0]])
    #     self.lambda_2.assign([result.x[1]])


    def train(self, nIter):
        optimizer = tf.keras.optimizers.Adam()
        for it in range(nIter):
            with tf.GradientTape() as tape:
                u_pred, v_pred, p_pred, f_u_pred, f_v_pred = self.net_NS(self.x, self.y, self.t)
                loss_value = self.loss(u_pred, v_pred, f_u_pred, f_v_pred)

            gradients = tape.gradient(loss_value, self.model.trainable_variables)

            optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))

            if it % 10 == 0:
                print("Iteration: {}, Loss: {:.4e}".format(it, loss_value.numpy()))

        self.lambda_1 = self.model.variables[0].numpy()
        self.lambda_2 = self.model.variables[1].numpy()


    def predict(self, x_star, y_star, t_star):

        u_star, v_star, p_star, _, _ = self.net_NS(x_star, y_star, t_star)

        return u_star, v_star, p_star


In [3]:
N_train = 5000
    
layers = [3, 20, 20, 20, 20, 20, 20, 20, 20, 2]

# Load Data
data = scipy.io.loadmat('./PINNs-master/main/Data/cylinder_nektar_wake.mat')
        
U_star = data['U_star'] # N x 2 x T
P_star = data['p_star'] # N x T
t_star = data['t'] # T x 1
X_star = data['X_star'] # N x 2

N = X_star.shape[0]
T = t_star.shape[0]

# Rearrange Data 
XX = np.tile(X_star[:,0:1], (1,T)) # N x T
YY = np.tile(X_star[:,1:2], (1,T)) # N x T
TT = np.tile(t_star, (1,N)).T # N x T

UU = U_star[:,0,:] # N x T
VV = U_star[:,1,:] # N x T
PP = P_star # N x T

x = XX.flatten()[:,None] # NT x 1
y = YY.flatten()[:,None] # NT x 1
t = TT.flatten()[:,None] # NT x 1

u = UU.flatten()[:,None] # NT x 1
v = VV.flatten()[:,None] # NT x 1
p = PP.flatten()[:,None] # NT x 1

In [4]:
idx = np.random.choice(N*T, N_train, replace=False)
x_train = x[idx,:]
y_train = y[idx,:]
t_train = t[idx,:]
u_train = u[idx,:]
v_train = v[idx,:]


In [17]:
model = PhysicsInformedNN(x_train, y_train, t_train, u_train, v_train, layers)

In [18]:
model.train(10)

Iteration: 0, Loss: 4.4366e+03


In [7]:
# Test Data
snap = np.array([100])
x_star = X_star[:,0:1]
y_star = X_star[:,1:2]
t_star = TT[:,snap]

u_star = U_star[:,0,snap]
v_star = U_star[:,1,snap]
p_star = P_star[:,snap]